# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [1]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [2]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3, os, sys
from sagemaker.pytorch import PyTorch

In [3]:
from sagemaker import get_execution_role
role = get_execution_role()
sess = sagemaker.Session()
sess.default_bucket(), sess.boto_region_name, get_execution_role()

('sagemaker-us-east-1-176302953206',
 'us-east-1',
 'arn:aws:iam::176302953206:role/service-role/AmazonSageMaker-ExecutionRole-20211110T114740')

In [4]:
bucket= 'image-classification-using-aws-sagemaker' 


In [5]:
os.environ["DEFAULT_S3_BUCKET"] = bucket
! echo "$DEFAULT_S3_BUCKET"

image-classification-using-aws-sagemaker


## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
#TODO: Fetch and upload the data to AWS S3
# Command to download and unzip data
#!wget -O https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
    
!wget -q https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip -O ./dogImages.zip

!unzip dogImages.zip

In [8]:
! echo s3://${DEFAULT_S3_BUCKET}/valid/

s3://image-classification-using-aws-sagemaker/valid/


In [ ]:
!aws s3 sync ./dogImages/valid  s3://${DEFAULT_S3_BUCKET}/valid/

In [ ]:
!aws s3 sync ./dogImages/test  s3://${DEFAULT_S3_BUCKET}/test/

In [ ]:
!aws s3 sync ./dogImages/train s3://${DEFAULT_S3_BUCKET}/train/

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [17]:
!pip install torch

In [18]:
!pip install torchvision

In [20]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [21]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "batch_size" : CategoricalParameter([32, 64, 128]),
    "learning_rate" : ContinuousParameter(0.001, 0.01),
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]


In [22]:
#TODO: Create estimators for your HPs

# TODO: Your estimator here
estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    #instance_type="ml.m5.large"
    instance_type="ml.p3.2xlarge",
)

# TODO: Your HP tuner here
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=6,
    max_parallel_jobs=2,
    objective_type=objective_type)

In [23]:
{"training": f"s3://{bucket}/"}

{'training': 's3://image-classification-using-aws-sagemaker/'}

In [24]:
# TODO: Fit your HP Tuner
tuner.fit({"training": f"s3://{bucket}/"}, wait=True) # TODO: Remember to include your data channels

.............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [25]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()#TODO

#Get the hyperparameters of the best trained model
#hyperparameters = best_estimator.hyperparameters()
#hyperparameters



hyperparameters = {"batch_size": int(best_estimator.hyperparameters()['batch_size'].replace('"', '')), \
                   "learning_rate": best_estimator.hyperparameters()['learning_rate']}
hyperparameters


2022-01-11 21:12:49 Starting - Preparing the instances for training
2022-01-11 21:12:49 Downloading - Downloading input data
2022-01-11 21:12:49 Training - Training image download completed. Training in progress.
2022-01-11 21:12:49 Uploading - Uploading generated training model
2022-01-11 21:12:49 Completed - Training job completed


{'batch_size': 32, 'learning_rate': '0.0025749942716753217'}

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [20]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig,
    rule_configs,
)
from sagemaker.debugger import ProfilerRule, ProfilerConfig, FrameworkProfile


In [25]:
# TODO: Set up debugging and profiling rules and hooks
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.exploding_tensor()),    
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.class_imbalance()), #specific to DL frameworks
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.OverallSystemUsage()), 
    ProfilerRule.sagemaker(rule_configs.CPUBottleneck()),    
    #ProfilerRule.sagemaker(rule_configs.OverallFrameworkMetrics()), #specific to DL frameworks   

    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]


hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "100",
        "eval.save_interval": "10"
    }
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=2)
)

debugger_config = DebuggerHookConfig(hook_parameters={"train.save_interval":"100", "eval.save_interval":"10"})

In [26]:
os.environ['SM_CHANNEL_TRAINING']=f's3://{bucket}/'
os.environ['SM_MODEL_DIR']=f's3://{bucket}/model/'
os.environ['SM_OUTPUT_DATA_DIR']=f's3://{bucket}/output/'

!echo $SM_CHANNEL_TRAINING
!echo $SM_MODEL_DIR
!echo $SM_OUTPUT_DATA_DIR

s3://image-classification-using-aws-sagemaker/
s3://image-classification-using-aws-sagemaker/model/
s3://image-classification-using-aws-sagemaker/output/


In [27]:
hyperparameters = {'batch_size': 32, 'learning_rate': '0.0025749942716753217'}
hyperparameters

{'batch_size': 32, 'learning_rate': '0.0025749942716753217'}

In [28]:
# TODO: Create and fit an estimator

# TODO: Your estimator here
estimator = PyTorch(
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    entry_point="train_model.py",
    framework_version="1.8",
    py_version="py36",
    hyperparameters=hyperparameters,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules,
)

estimator.fit({"training": f"s3://{bucket}/"}, wait=True)

2022-01-12 13:37:08 Starting - Starting the training job...
2022-01-12 13:37:12 Starting - Launching requested ML instancesLossNotDecreasing: InProgress
VanishingGradient: InProgress
ExplodingTensor: InProgress
Overfit: InProgress
ClassImbalance: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LowGPUUtilization: InProgress
OverallSystemUsage: InProgress
CPUBottleneck: InProgress
ProfilerReport: InProgress
......
2022-01-12 13:38:35 Starting - Preparing the instances for training.........
2022-01-12 13:40:09 Downloading - Downloading input data............
2022-01-12 13:41:57 Training - Downloading the training image...........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-01-12 13:43:53,021 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-01-12 13:43:53,045 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2

UnexpectedStatusException: Error for Training job pytorch-training-2022-01-12-13-37-08-137: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 train_model.py --batch_size 32 --learning_rate 0.0025749942716753217"
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
  0%|          | 0.00/97.8M [00:00<?, ?B/s]  6%|â         | 6.20M/97.8M [00:00<00:01, 64.9MB/s] 13%|ââ        | 12.7M/97.8M [00:00<00:01, 66.6MB/s] 20%|ââ        | 19.1M/97.8M [00:00<00:01, 66.8MB/s] 26%|âââ       | 25.5M/97.8M [00:00<00:01, 67.2MB/s] 33%|ââââ      | 32.0M/97.8M [00:00<00:01, 67.3MB/s] 39%|ââââ      | 38.5M/97.8M [00:00<00:00, 67.5MB/s] 46%|âââââ     | 44.9M/97.8M [00:00<00:00, 66.2MB/s] 52%|ââââââ    | 51.2M/97.8M [00:00<00:00, 64.4MB/s] 59%|ââââââ    | 57.4M/97.8M [00:00<00:00, 63.9MB/s] 65%|âââââââ 

In [1]:
# TODO: Plot a debugging output.

from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())


[2022-01-12 00:09:52.553 ip-172-16-95-178:18781 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


NameError: name 'estimator' is not defined

In [40]:
trial.tensor_names()

[2022-01-11 22:37:00.129 ip-172-16-141-130:19858 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2022-01-11 22:37:01.150 ip-172-16-141-130:19858 INFO trial.py:210] Loaded all steps


['CrossEntropyLoss_output_0',
 'gradient/ResNet_fc.0.bias',
 'gradient/ResNet_fc.0.weight',
 'gradient/ResNet_fc.2.bias',
 'gradient/ResNet_fc.2.weight',
 'layer1.0.relu_input_0',
 'layer1.0.relu_input_1',
 'layer1.0.relu_input_2',
 'layer1.1.relu_input_0',
 'layer1.1.relu_input_1',
 'layer1.1.relu_input_2',
 'layer1.2.relu_input_0',
 'layer1.2.relu_input_1',
 'layer1.2.relu_input_2',
 'layer2.0.relu_input_0',
 'layer2.0.relu_input_1',
 'layer2.0.relu_input_2',
 'layer2.1.relu_input_0',
 'layer2.1.relu_input_1',
 'layer2.1.relu_input_2',
 'layer2.2.relu_input_0',
 'layer2.2.relu_input_1',
 'layer2.2.relu_input_2',
 'layer2.3.relu_input_0',
 'layer2.3.relu_input_1',
 'layer2.3.relu_input_2',
 'layer3.0.relu_input_0',
 'layer3.0.relu_input_1',
 'layer3.0.relu_input_2',
 'layer3.1.relu_input_0',
 'layer3.1.relu_input_1',
 'layer3.1.relu_input_2',
 'layer3.2.relu_input_0',
 'layer3.2.relu_input_1',
 'layer3.2.relu_input_2',
 'layer3.3.relu_input_0',
 'layer3.3.relu_input_1',
 'layer3.3.rel

In [41]:
len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN))

TensorUnavailable: Tensor nll_loss_output_0 was not saved.

In [42]:
len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL))

TensorUnavailable: Tensor nll_loss_output_0 was not saved.

In [43]:
def get_data(trial, tname, mode):
    tensor = trial.tensor(tname)
    steps = tensor.steps(mode=mode)
    vals = []
    for s in steps:
        vals.append(tensor.value(s, mode=mode))
    return steps, vals



In [44]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot


def plot_tensor(trial, tensor_name):

    steps_train, vals_train = get_data(trial, tensor_name, mode=ModeKeys.TRAIN)
    print("loaded TRAIN data")
    steps_eval, vals_eval = get_data(trial, tensor_name, mode=ModeKeys.EVAL)
    print("loaded EVAL data")

    fig = plt.figure(figsize=(10, 7))
    host = host_subplot(111)

    par = host.twiny()

    host.set_xlabel("Steps (TRAIN)")
    par.set_xlabel("Steps (EVAL)")
    host.set_ylabel(tensor_name)

    (p1,) = host.plot(steps_train, vals_train, label=tensor_name)
    print("completed TRAIN plot")
    (p2,) = par.plot(steps_eval, vals_eval, label="val_" + tensor_name)
    print("completed EVAL plot")
    leg = plt.legend()

    host.xaxis.get_label().set_color(p1.get_color())
    leg.texts[0].set_color(p1.get_color())

    par.xaxis.get_label().set_color(p2.get_color())
    leg.texts[1].set_color(p2.get_color())

    plt.ylabel(tensor_name)

    plt.show()

In [45]:
plot_tensor(trial, "CrossEntropyLoss_output_0")

TensorUnavailable: Tensor nll_loss_output_0 was not saved.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

In [ ]:
! aws s3 ls {rule_output_path} --recursive

In [ ]:
! aws s3 cp {rule_output_path} ./ --recursive

In [ ]:
# TODO: Display the profiler output
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
rule_output_path

In [46]:
# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

import IPython

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

FileNotFoundError: [Errno 2] No such file or directory: 'ProfilerReport/profiler-output/profiler-report.html'

## Model Deploying

In [48]:
# TODO: Deploy your model to an endpoint

# TODO: Add your deployment configuration like instance type and number of instances
predictor=estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium') 


---------------!

In [50]:
# TODO: Run an prediction on the endpoint

import requests
request_dict={ "url": "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/20113314/Carolina-Dog-standing-outdoors.jpg" }

image = requests.get(request_dict['url']).content# TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image, initial_args={"ContentType": "image/jpeg"})
response

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-training-2022-01-11-22-39-38-854 in account 176302953206 for more information.

In [51]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()